### Guided Practice: Logit Function and Odds

In [2]:
import numpy as np

def logit_func(odds):
    # uses a float (odds) and returns back the log odds (logit)
    #logit function
    return np.log(odds)

def sigmoid_func(logit):
    # uses a float (logit) and returns back the probability
    
     return 1. / (1 + np.exp(-logit))

odds_set = [
    5./1,
    20./1,
    1.1/1,
    1.8/1,
    1.6/1
]

In [3]:
for odds in odds_set:
    log_odds = logit_func(odds)
    prob = sigmoid_func(log_odds)
    print prob *100
    

83.3333333333
95.2380952381
52.380952381
64.2857142857
61.5384615385


In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [5]:
lm = LogisticRegression()

df = pd.read_csv('../../assets/dataset/collegeadmissions.csv')

In [6]:
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [7]:
dummies = pd.get_dummies(df['rank'])
df = df.join(dummies)

In [8]:
df.head()

,admit,gre,gpa,rank,1,2,3,4
0,0,380,3.61,3,0.0,0.0,1.0,0.0
1,1,660,3.67,3,0.0,0.0,1.0,0.0
2,1,800,4.00,1,1.0,0.0,0.0,0.0
3,1,640,3.19,4,0.0,0.0,0.0,1.0
4,0,520,2.93,4,0.0,0.0,0.0,1.0


In [9]:
X = df.ix[:,1:7]
y = df.admit

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 


In [10]:
lm.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
import numpy as np
import pandas as pd

In [12]:
print lm.coef_
print lm.intercept_
print df.admit.mean()

[[ 0.0010736   0.48538522 -0.68443776 -0.65513022 -0.35699227 -0.60751395]]
[-0.99283173]
0.3175


Below is some code to walk through confusion matrices. It'll be useful for working through the Titanic problem.

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

Below the ROC curve is based on various thresholds: it shows with a false positive rate (x-axis) ~0, it also expects a true positive rate (y-axis) ~0 (the same, ish, for the top right hand of the figure).

The second chart, which does not play with thesholds, shows the one true TPR and FPR point, joined to 0,0 and 1,1.

The first chart will be more effective as you compare models and determine where the decision line should exist for the data. The second simplifies the first in case this idea of thresholds is confusing.

In [16]:
predicted = lm.predict(X_test) 
print predicted

probas = lm.predict_proba(X_test)

print probas[:,1]


fpr = roc_curve(y_test, probas[:,1])[1]
tpr = roc_curve(y_test, probas[:,1])[0]

#plt.plot(fpr, tpr)

#plt.plot(roc_curve(y_test, probas[:,1])[0], roc_curve(y_test, probas[:,1])[1])

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 0]
[ 0.4019947   0.47559369  0.29875477  0.21166323  0.33763971  0.20270549
  0.14534323  0.41571341  0.17411835  0.50750608  0.48388898  0.48533947
  0.18182052  0.17224536  0.3903838   0.36689201  0.17590001  0.42773056
  0.19506435  0.39756472  0.29875477  0.18715494  0.46833594  0.18883776
  0.18340467  0.1712393   0.2656542   0.23472591  0.17551914  0.47680441
  0.47214788  0.17078413  0.32797676  0.13959513  0.33846822  0.36083667
  0.46871675  0.21814506  0.42162011  0.21917103  0.39056004  0.28543679
  0.19441931  0.57517419  0.29598239  0.39287313  0.40020797  0.21240554
  0.15217619  0.45141758  0.39733634  0.38056226  0.45030941  0.26405241
  0.42978585  0.1425155   0.24777978  0.22540048  0.21147117  0.47024147
  0.26945899  0.35908509 

In [17]:
predictionDf = pd.DataFrame({"actual": y_test, "predicted": np.array(probas[:,1])})
predictionDf


# for threshold in [0.1, 0.2,0.3,.4,.5]:
#     columnName = "predicted class_{}".format(threshold)
#     predictionDf[columnName] = predictionDf.predicted.apply(
#         lambda prob: 1 if prob >= threshold else 0)
#     print pd.crosstab(predictionDf.actual, predictionDf[columnName])

,actual,predicted
209,0,0.401995
280,0,0.475594
33,1,0.298755
210,0,0.211663
93,0,0.337640
84,1,0.202705
329,0,0.145343
94,1,0.415713
266,0,0.174118
126,1,0.507506


In [ ]:
plt.plot(roc_curve(y_test, actuals)[0], roc_curve(y_test, actuals)[1])

Finally, you can use the `roc_auc_score` function to calculate the area under these curves (AUC).

In [ ]:
roc_auc_score(df['admit'], lm.predict(feature_set)

### Titanic Problem

** Goals **

1. Spend a few minutes determining which data would be most important to use in the prediction problem. You may need to create new features based on the data available. Consider using a feature selection aide in sklearn. But a worst case scenario; identify one or two strong features that would be useful to include in the model.
2. Spend 1-2 minutes considering which _metric_ makes the most sense to optimize. Accuracy? FPR or TPR? AUC? Given the business problem (understanding survival rate aboard the Titanic), why should you use this metric?
3. Build a tuned Logistic model. Be prepared to explain your design (including regularization), metric, and feature set in predicting survival using the tools necessary (such as a fit chart).